In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math
import numpy as np

# Hi, you must have come from UCSB_Loads and usage and must be in the year of decarb :) Run this file now.


## STATIC INPUTS


In [2]:
# #import Building Meta Data
# buildingMetaData_df = pd.read_excel('Building_MetaData.xlsx')
# buildingMetaData_df.head()

## Import inputs from inputs.py


In [3]:
from districtsystem.inputs import (
    CUP_inputs,
    building_inputs,
    building_inputs_df,
    hotWater_schedule3_inputs_df,
    domesticWater_schedule4_inputs_df,
    chilledCTOcean_schedule5_inputs_df,
    weather_df,
    buildingModule_inputs,
    districtModule_inputs,
    hotWaterMonths,
    dateTime,
)

## Building module outputs


#### Loops through every building


In [4]:
# defining inputs ---> static-same for all buildings
HW_LoopSTP = districtModule_inputs["Loop HW STP (°F)"]
HW_supplyLosses = CUP_inputs["Setpoints"]["Hot Water Supply Losses"]
HW_returnLosses = CUP_inputs["Setpoints"]["Hot Water Return Losses"]
CHW_LoopSTP = districtModule_inputs["Loop CHW STP (°F)"]
CHW_supplyLosses = CUP_inputs["Setpoints"]["Chilled Water Supply Losses"]
CHW_deltaT_Max = building_inputs["Chilled Water"]["Delta T @ Max Load"]
CHW_deltaT_Min = building_inputs["Chilled Water"]["Delta T @ Min Load"]
# CHW_maxLoad= building_inputs["Chilled Water"]["Max Load"]
HHW_BldgSTP = buildingModule_inputs["Building HHW STP (°F)"]
HHW_supply_Temps = building_inputs_df.loc[
    building_inputs_df["Building Level Categories"]
    == "Return HHW Temps given Supply Temp",
    "Building Level User Inputs",
]
HHW_return_Temps = building_inputs_df.loc[
    building_inputs_df["Building Level Categories"]
    == "Return HHW Temps given Supply Temp",
    "Building Level Values",
]
DHWMonths = domesticWater_schedule4_inputs_df["Month"]
DHWSetpoint = domesticWater_schedule4_inputs_df["Building"]
# DHWmaxLoad = building_inputs["Domestic Hot Water"]["Max Load"]
DHWmaxApproach = building_inputs["Domestic Hot Water"]["Max Approach"]
DHWminApproach = building_inputs["Domestic Hot Water"]["Min Approach"]
# DHWloadMinApproach = building_inputs["Domestic Hot Water"]["Load @ Min Approach"]

In [5]:
# Reset the index to be sequential (0, 1, 2, 3, ...)
HHW_return_Temps = HHW_return_Temps.reset_index(drop=True)
# data type set to int
HHW_supply_Temps = HHW_supply_Temps.astype(int)

In [6]:
from districtsystem.building_staticInputs import BuildingParameters

# creating instance by assigning static parameters
params_bldg = BuildingParameters(
    HW_LoopSTP=HW_LoopSTP,
    HW_supplyLosses=HW_supplyLosses,
    CHW_LoopSTP=CHW_LoopSTP,
    CHW_supplyLosses=CHW_supplyLosses,
    CHW_deltaT_Max=CHW_deltaT_Max,
    CHW_deltaT_Min=CHW_deltaT_Min,
    HHW_supply_Temps=HHW_supply_Temps,
    HHW_return_Temps=HHW_return_Temps,
    HHW_BldgSTP=HHW_BldgSTP,
    DHWMonths=DHWMonths,
    DHWSetpoint=DHWSetpoint,
    buildingDate=weather_df["Date"],
    hotWaterMonths=hotWaterMonths,
    DHWmaxApproach=DHWmaxApproach,
    DHWminApproach=DHWminApproach,
    HW_returnLosses=HW_returnLosses,
)

In [7]:
# Load the DataFrame from Calculation Maps ( CUP thermal loads for this CUP module)
new_CUP_Therm_Loads_df = pd.read_pickle("inputs/new_CUP_Therm_Loads.pkl")

# Now 'df' contains the DataFrame loaded from the pickle file
new_CUP_Therm_Loads_df.loc[:, :]  # Verify the DataFrame

,cooling CAAN,Current New CUP Cooling Load (kBtu),heating CAAN,Current New CUP Heating Load (kBtu),hot water CAAN,Current New CUP Hot Water Load (kBtu),cooking CAAN,Current New CUP Cooking Load (kBtu),laundry CAAN,Current New CUP Laundry Load (kBtu),Building ID CAAN,Ambient Air Wet Bulb Temp (F),Timestamp
0,8517,0.0,8517,11.360497,8517,0.000000,8517,0.0,8517,0.000000,8517,49.501190,2010-01-01 00:00:00
1,8517,0.0,8517,13.899768,8517,0.001166,8517,0.0,8517,0.000000,8517,49.524185,2010-01-01 01:00:00
2,8517,0.0,8517,16.125689,8517,0.000000,8517,0.0,8517,0.000000,8517,49.515631,2010-01-01 02:00:00
3,8517,0.0,8517,17.244490,8517,0.001166,8517,0.0,8517,0.000000,8517,48.960694,2010-01-01 03:00:00
4,8517,0.0,8517,15.584648,8517,0.001166,8517,0.0,8517,0.000000,8517,48.592221,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313995,8832,0.0,8832,3.377490,8832,0.033585,8832,0.0,8832,0.001875,8832,57.984365,2010-12-31 19:00:00
1313996,8832,0.0,8832,3.110924,8832,0.034892,8832,0.0,8832,0.000000,8832,57.984365,2010-12-31 20:00:00
1313997,8832,0.0,8832,3.194271,8832,0.035576,8832,0.0,8832,0.000286,8832,57.984365,2010-12-31 21:00:00
1313998,8832,0.0,8832,4.936392,8832,0.035084,8832,0.0,8832,0.000000,8832,57.984365,2010-12-31 22:00:00


In [8]:
from districtsystem.building_module import Building

allBldgModule_output = pd.DataFrame()
# Loop through all CAANs in CUP thermal loads input df
for caan in new_CUP_Therm_Loads_df["Building ID CAAN"].unique():
    coolingLoad = (
        new_CUP_Therm_Loads_df.loc[
            new_CUP_Therm_Loads_df["Building ID CAAN"] == caan,
            "Current New CUP Cooling Load (kBtu)",
        ]
        * 1000
    )
    heatingLoad = (
        new_CUP_Therm_Loads_df.loc[
            new_CUP_Therm_Loads_df["Building ID CAAN"] == caan,
            "Current New CUP Heating Load (kBtu)",
        ]
        * 1000
    )
    DHWLoad = (
        new_CUP_Therm_Loads_df.loc[
            new_CUP_Therm_Loads_df["Building ID CAAN"] == caan,
            "Current New CUP Hot Water Load (kBtu)",
        ]
        * 1000
    )

    coolingLoad = coolingLoad.reset_index(drop=True)
    heatingLoad = heatingLoad.reset_index(drop=True)
    DHWLoad = DHWLoad.reset_index(drop=True)

    # building Chilled water max
    CHW_maxLoad = coolingLoad.max()

    # building DHW max
    DHW_maxLoad = DHWLoad.max() or np.nan

    DHW_loadMinApproach = DHW_maxLoad * 0.26

    # creating building instance by  assinging changing / calculated parameters(fields) and calculating outputs
    bldg = Building(
        parameters=params_bldg,
        caan_no=caan,
        name="Lab",
        coolingLoad=coolingLoad,
        heatingLoad=heatingLoad,
        DHWLoad=DHWLoad,
        timeStamp=dateTime,
        CHW_maxLoad=CHW_maxLoad,
        DHW_maxLoad=DHW_maxLoad,
        DHW_loadMinApproach=DHW_loadMinApproach,
    )
    buildingModule_output = bldg.compute()
    allBldgModule_output = pd.concat(
        [allBldgModule_output, buildingModule_output], ignore_index=True
    )  # Concatenate new row to existing DataFrame

allBldgModule_output.head(10)

/Users/samueld/PycharmProjects/district-system/districtsystem/building_module.py:154: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dhwrt.fillna(0, inplace=True)
/Users/samueld/PycharmProjects/district-system/districtsystem/building_module.py:154: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dhwrt.fillna(0, inplace=True)
/Users/samueld/PycharmProjects/district-system/districtsystem/building_module.py:154: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects

,caan_no,Time Stamp,Space Heating Load (Btu/h),DHW Load (Btu/h),Cooling Load (Btu/h),Loop HWST @ Building (°F),Loop CHWST @ Building (°F),CHWRT (°F),CHWR Flow (gpm),Building HHWRT (°F),HHWRflow,Building Domestic Water Temp (°F),Building DHWRT (°F),Building DHWR Flow (gpm),District HWS Flow (gpm),Bypassed Return to HHWS (gpm),District HWR Flow (gpm),District HWRT (°F),Check Building HWS = HWR
0,8517,2010-01-01 00:00:00,11360.497415,0.00000,0.0,165,46.0,54.0,0.0,110,1.13605,55,60.0,0.000000,0.413109,0.722941,0.413109,107.0,True
1,8517,2010-01-01 01:00:00,13899.768428,1.16629,0.0,165,46.0,54.0,0.0,110,1.389977,55,60.0,0.000022,0.505468,0.884531,0.505468,106.997803,True
2,8517,2010-01-01 02:00:00,16125.688563,0.00000,0.0,165,46.0,54.0,0.0,110,1.612569,55,60.0,0.000000,0.586389,1.02618,0.586389,107.0,True
3,8517,2010-01-01 03:00:00,17244.489724,1.16629,0.0,165,46.0,54.0,0.0,110,1.724449,55,60.0,0.000022,0.627095,1.097377,0.627095,106.998229,True
4,8517,2010-01-01 04:00:00,15584.648346,1.16629,0.0,165,46.0,54.0,0.0,110,1.558465,55,60.0,0.000022,0.566737,0.99175,0.566737,106.99804,True
5,8517,2010-01-01 05:00:00,14516.358026,0.00000,0.0,165,46.0,54.0,0.0,110,1.451636,55,60.0,0.000000,0.527868,0.923768,0.527868,107.0,True
6,8517,2010-01-01 06:00:00,17501.219754,1.16629,0.0,165,46.0,54.0,0.0,110,1.750122,55,60.0,0.000022,0.636430,1.113714,0.63643,106.998255,True
7,8517,2010-01-01 07:00:00,18265.328215,1.16629,0.0,165,46.0,54.0,0.0,110,1.826533,55,60.0,0.000022,0.664216,1.162339,0.664216,106.998328,True
8,8517,2010-01-01 08:00:00,2188.605181,0.00000,0.0,165,46.0,54.0,0.0,110,0.218861,55,60.0,0.000000,0.079586,0.139275,0.079586,107.0,True
9,8517,2010-01-01 09:00:00,282.020111,1.16629,0.0,165,46.0,54.0,0.0,110,0.028202,55,60.0,0.000022,0.010277,0.017947,0.010277,106.891924,True


In [9]:
# No. of bldgs going ahead in the cup =  divide the first number that prints below by 8760 to know the number of bldgs
allBldgModule_output.shape

(1314000, 19)

In [10]:
allBldgModule_output.iloc[:, :]

,caan_no,Time Stamp,Space Heating Load (Btu/h),DHW Load (Btu/h),Cooling Load (Btu/h),Loop HWST @ Building (°F),Loop CHWST @ Building (°F),CHWRT (°F),CHWR Flow (gpm),Building HHWRT (°F),HHWRflow,Building Domestic Water Temp (°F),Building DHWRT (°F),Building DHWR Flow (gpm),District HWS Flow (gpm),Bypassed Return to HHWS (gpm),District HWR Flow (gpm),District HWRT (°F),Check Building HWS = HWR
0,8517,2010-01-01 00:00:00,11360.497415,0.000000,0.0,165,46.0,54.0,0.0,110,1.13605,55,60.000000,0.000000,0.413109,0.722941,0.413109,107.0,True
1,8517,2010-01-01 01:00:00,13899.768428,1.166290,0.0,165,46.0,54.0,0.0,110,1.389977,55,60.000000,0.000022,0.505468,0.884531,0.505468,106.997803,True
2,8517,2010-01-01 02:00:00,16125.688563,0.000000,0.0,165,46.0,54.0,0.0,110,1.612569,55,60.000000,0.000000,0.586389,1.02618,0.586389,107.0,True
3,8517,2010-01-01 03:00:00,17244.489724,1.166290,0.0,165,46.0,54.0,0.0,110,1.724449,55,60.000000,0.000022,0.627095,1.097377,0.627095,106.998229,True
4,8517,2010-01-01 04:00:00,15584.648346,1.166290,0.0,165,46.0,54.0,0.0,110,1.558465,55,60.000000,0.000022,0.566737,0.99175,0.566737,106.99804,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313995,8832,2010-12-31 19:00:00,3377.490071,33.585448,0.0,165,46.0,54.0,0.0,110,0.337749,55,71.544665,0.000719,0.123537,0.214931,0.123537,106.776263,True
1313996,8832,2010-12-31 20:00:00,3110.923947,34.892335,0.0,165,46.0,54.0,0.0,110,0.311092,55,72.198973,0.000752,0.113876,0.197968,0.113876,106.750381,True
1313997,8832,2010-12-31 21:00:00,3194.270926,35.575981,0.0,165,46.0,54.0,0.0,110,0.319427,55,72.541249,0.000770,0.116925,0.203272,0.116925,106.753461,True
1313998,8832,2010-12-31 22:00:00,4936.391524,35.084471,0.0,165,46.0,54.0,0.0,110,0.493639,55,72.295168,0.000757,0.180262,0.314134,0.180262,106.84168,True


In [11]:
# #filtered district building meta data
# districtMetaData_df = buildingMetaData_df[buildingMetaData_df["Currently on District System (Y/N)"]=="Y"]

# districtMetaData_df.head()

In [12]:
# from districtsystem.building_module import Building
# #path to folder that contains all building calibrated data files
# building_usage_folder =  r"C:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\Building usage data"
# # Get a list of all files in the folder
# files = os.listdir(building_usage_folder)
# allBldgModule_output =pd.DataFrame()
# #loop through all files
# for file in files:
#     print(file)
#     file_path = os.path.join(building_usage_folder, file)
#     buildingMockUsage_df = pd.read_excel(file_path)
#     if buildingMockUsage_df['CAAN_Number'][0] in districtMetaData_df['CAAN_Number'].values:

#         #building Chilled water
#         CHW_maxLoad = buildingMockUsage_df['Cooling Load (Btu/h)'].max()


#         #building DHW
#         DHW_maxLoad = buildingMockUsage_df['DHW Load (Btu/h)'].max()
#         DHW_loadMinApproach= DHW_maxLoad*0.26

#         # #building HHW
#         # maxValue_HHW = buildingMockUsage_df['Heating Load (Btu/h)'].max()
#         # HHW_maxLoad= maxValue_HHW + DHW_maxLoad

#         # creating building instance by  assinging changing / calculated parameters(fields) and calculating outputs
#         bldg = Building(parameters=params_bldg, caan_no=buildingMockUsage_df['CAAN_Number'][0], name="Lab",coolingLoad = buildingMockUsage_df['Cooling Load (Btu/h)'],\
#                         heatingLoad =buildingMockUsage_df['Heating Load (Btu/h)'],DHWLoad =buildingMockUsage_df['DHW Load (Btu/h)'],\
#                         timeStamp = buildingMockUsage_df['Date and Time'],CHW_maxLoad=CHW_maxLoad,DHW_maxLoad=DHW_maxLoad,DHW_loadMinApproach=DHW_loadMinApproach)
#         buildingModule_output = bldg.compute()
#         allBldgModule_output = pd.concat([allBldgModule_output, buildingModule_output], ignore_index=True)  # Concatenate new row to existing DataFrame

# allBldgModule_output.head(10)

In [13]:
# Convert 'Time Stamp' column to datetime format
allBldgModule_output["Time Stamp"] = pd.to_datetime(allBldgModule_output["Time Stamp"])

# Group by 'Time Stamp' and sum the 'Loop HWST @ Building (°F)' column
sum_by_timestamp = allBldgModule_output.groupby("Time Stamp")[
    "Loop HWST @ Building (°F)"
].sum()
print(sum_by_timestamp.shape)

(8760,)


## Building Mixing


In [14]:
from districtsystem.building_mixing import building_mixing

bldgs_mixed = building_mixing(df=allBldgModule_output, parameters=params_bldg)
buildingMixing_output = bldgs_mixed.compute()
buildingMixing_output.head()

ck Time Stamp
2010-01-01 00:00:00    666.026494
2010-01-01 01:00:00    717.288476
2010-01-01 02:00:00    736.936338
2010-01-01 03:00:00    738.476758
2010-01-01 04:00:00    673.297155
                          ...    
2010-12-31 19:00:00    409.427945
2010-12-31 20:00:00    492.481079
2010-12-31 21:00:00    531.998901
2010-12-31 22:00:00    599.510279
2010-12-31 23:00:00    630.532248
Name: District HWS Flow (gpm), Length: 8760, dtype: float64 0       170
1       170
2       170
3       170
4       170
       ... 
8755    170
8756    170
8757    170
8758    170
8759    170
Name: Loop HW STP (°F), Length: 8760, dtype: int64 0       106.986043
1       106.987195
2       106.989673
3       106.989115
4         106.9905
           ...    
8755    106.936625
8756    106.948661
8757    106.954629
8758    106.962543
8759    106.968389
Length: 8760, dtype: object


,Total Space Heating Load (Btu/h),Total DHW Load (Btu/h),District HWS Flow (gpm),District HWRT (°F),District CHWS Flow (gpm),District CHWRT (°F),Total Heating Load (Btu/h),Total Cooling Load (Btu/h)
0,3.943126e+07,9772.710226,666.026494,106.986043,15.241074,54.890725,20984482.426162,82993.175602
1,4.216031e+07,9699.698364,717.288476,106.987195,14.256731,54.827249,22599179.545175,77180.590230
2,4.299871e+07,7990.915363,736.936338,106.989673,14.026118,54.812509,23217299.856401,75828.765272
3,4.313862e+07,8440.542019,738.476758,106.989115,14.206786,54.824053,23266037.185625,76887.501990
4,3.924532e+07,6716.047942,673.297155,106.9905,17.355616,54.870586,21212058.510258,94332.859924


In [15]:
buildingMixing_output.iloc[:, :]

,Total Space Heating Load (Btu/h),Total DHW Load (Btu/h),District HWS Flow (gpm),District HWRT (°F),District CHWS Flow (gpm),District CHWRT (°F),Total Heating Load (Btu/h),Total Cooling Load (Btu/h)
0,3.943126e+07,9772.710226,666.026494,106.986043,15.241074,54.890725,20984482.426162,82993.175602
1,4.216031e+07,9699.698364,717.288476,106.987195,14.256731,54.827249,22599179.545175,77180.590230
2,4.299871e+07,7990.915363,736.936338,106.989673,14.026118,54.812509,23217299.856401,75828.765272
3,4.313862e+07,8440.542019,738.476758,106.989115,14.206786,54.824053,23266037.185625,76887.501990
4,3.924532e+07,6716.047942,673.297155,106.9905,17.355616,54.870586,21212058.510258,94332.859924
...,...,...,...,...,...,...,...,...
8755,2.797810e+07,30347.526483,409.427945,106.936625,22.274893,54.892898,12909954.089652,121319.068967
8756,3.321937e+07,29783.701864,492.481079,106.948661,16.201939,54.918414,15525795.708985,88449.743048
8757,3.579863e+07,28548.608956,531.998901,106.954629,15.047130,54.878147,16770034.109234,81842.443705
8758,3.788561e+07,26546.610880,599.510279,106.962543,15.075032,54.879954,18895801.857486,82007.828775


In [16]:
buildingMixing_output.to_pickle("outputs/buildingMixing_output.pkl")

# Now, lets go to UCSB_CUP Heat pump TES
